## Harvard's Physical Impact
How much of the United States does Harvard own?

In [1]:
import cityscraper
import numpy as np
import pandas as pd
import math

In [2]:
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

There are gaps in the available information for cities abroad, so this analysis will focus on Harvard's national impact. Hence, the buildings abroad can be ignored.

In [3]:
buildings = pd.read_excel("Spreadsheets/2018_building_reference_list.xlsx", header=3)
buildings = buildings[~buildings['City'].isin(['Sardis', 'Fiesole', 'Florence', 'Shanghai', 'London'])]
footage = buildings['GSF SF Total'].sum()
count = buildings['GSF SF Total'].count()

Harvard owns {{count}} buildings in the following cities in the United States: Jamaica Plain, Roslindale, Cambridge, Washington DC, Ledyard, Allston, Hamilton, Petersham, Harvard, Bedford, Southborough, Shrewsbury, Kittery Point, Boston, Los Angeles, San Francisco, Holyoke, Somerville, Menlo Park, Burlingame, Lowell and New York.

Most of the data available on cities can be found at http://www.city-data.com, so I built a webscraper, `cityscraper.py`, to be able to pull data from every city that Harvard has buildings in.

In [4]:
df = pd.read_excel("Spreadsheets/city_url.xlsx", header=0)
results = []

for i in range(df['Type'].count()):
    city = df.iloc[[i]]['City'].values[0]
    my_type = df.iloc[[i]]['Type'].values[0]
    endpoint = df.iloc[[i]]['Endpoint'].values[0]

    results.append(cityscraper.scraper(city, my_type, endpoint))

city_data = pd.DataFrame(results)

To facilitate comparison, the `Area` column of the `city_data` dataframe will be converted from square miles to square feet, by multiplying by $27878000$.

In [5]:
city_data['Square Footage'] = city_data['Area'] * 27878000

Now, we can easily determine how much of a given city Harvard owns.

In [6]:
footage_results = []

for i in range(city_data['City'].count()):
    city = city_data.iloc[[i]]['City'].values[0]
    city_buildings = buildings[buildings['City'] == city]
    harv_footage = city_buildings['GSF SF Total'].sum()
    city_footage = city_data.iloc[i]['Square Footage']
    perc_harv = (harv_footage / city_footage) * 100
    footage_results.append({'City': city, 'Harvard Footage': harv_footage, 'City Footage': city_footage, 'Percentage Harvard': perc_harv})

final = pd.DataFrame(footage_results)
final.sort_values('Percentage Harvard', ascending=False, inplace=True)
final.reset_index(inplace=True, drop=True)

In [7]:
print(footage_results)

[{'City': 'Jamaica Plain', 'Harvard Footage': 122906.0, 'City Footage': 88512650.0, 'Percentage Harvard': 0.13885698823840434}, {'City': 'Roslindale', 'Harvard Footage': 1681.0, 'City Footage': 102758308.0, 'Percentage Harvard': 0.0016358774611197374}, {'City': 'Cambridge', 'Harvard Footage': 17151456.3, 'City Footage': 179255540.0, 'Percentage Harvard': 9.5681596786353165}, {'City': 'Washington', 'Harvard Footage': 256038.0, 'City Footage': 1711709200.0, 'Percentage Harvard': 0.014958031422627163}, {'City': 'Ledyard', 'Harvard Footage': 21284.0, 'City Footage': 1062151800.0, 'Percentage Harvard': 0.0020038566992025058}, {'City': 'Allston', 'Harvard Footage': 5777374.0, 'City Footage': 101420164.0, 'Percentage Harvard': 5.6964747168028635}, {'City': 'Hamilton', 'Harvard Footage': 27809.0, 'City Footage': 407018800.0, 'Percentage Harvard': 0.0068323625346052817}, {'City': 'Petersham', 'Harvard Footage': 84358.0, 'City Footage': 1510987600.0, 'Percentage Harvard': 0.0055829710316616758},

The three cities that Harvard owns most of are:
1. Cambridge, owning {{math.ceil(final.iloc[0]['Percentage Harvard'] * 100) / 100}}% of the land.
1. Allston, owning {{math.ceil(final.iloc[1]['Percentage Harvard'] * 100) / 100}}% of the land.
1. Boston, owning {{math.ceil(final.iloc[2]['Percentage Harvard'] * 100) / 100}}% of the land.

In [8]:
labels1 = ['Cambridge','Harvard']
labels2 = ['Allston', 'Harvard']
labels3 = ['Boston', 'Harvard']
values1 = [100 - 9.57, 9.57]
values2 = [100 - 5.7, 5.7]
values3 = [100 - 0.28, 0.28]

trace1 = go.Pie(labels=labels1, values=values1)
trace2 = go.Pie(labels=labels2, values=values2)
trace3 = go.Pie(labels=labels3, values=values3)

fig = {
    'data': [
        {
            'labels': labels1,
            'values': values1,
            'type': 'pie',
            'name': 'Cambridge',
            'marker': {'colors': ['rgb(46, 204, 113)',
                                  '#D2232A']},
            'domain': {'x': [0, .32]},
            'hole': .4,
            'hoverinfo':'percent+name',
            'textinfo':'none'
        },
        {
            'labels': labels2,
            'values': values2,
            'type': 'pie',
            'name': 'Allston',
            'marker': {'colors': ['rgb(52, 73, 94)',
                                  '#D2232A']},
            'domain': {'x': [.33, .65]},
            'hole': .4,
            'hoverinfo':'percent+name',
            'textinfo':'none'

        },
        {
            'labels': labels3,
            'values': values3,
            'type': 'pie',
            'name': 'Boston',
            'marker': {'colors': ['rgb(241, 196, 15)',
                                  '#D2232A']},
            'domain': {'x': [.66, 1]},
            'hole': .4,
            'hoverinfo':'percent+name',
            'textinfo':'none'
        }
    ],
    'layout': {'title': "Harvard's Presence In Cities",
               "font" : dict(family='Gotham', size=18),
              "annotations": [
                {
                    "font": {
                        "size": 40,
                        "family": "Gotham"
                    },
                    "showarrow": False,
                    "text": "C",
                    "x": 0.143,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 40,
                        "family": "Gotham"
                    },
                    "showarrow": False,
                    "text": "A",
                    "x": 0.49,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 40,
                        "family": "Gotham"
                    },
                    "showarrow": False,
                    "text": "B",
                    "x": 0.85,
                    "y": 0.5
                }
        ]}
}


iplot(fig)